# 가장 좋은 결과를 낼 수 있는 feature항목 추출

## 모든 feature를 사용한 결과와, 선택 추출된 feature만 사용한 결과 정확도에 차이가 남
#### logistic 회귀 이용하여 coef_ 항목에서 영향력이 높은 feature를 선택. 최적의 갯수 선택

### 데이터 준비하기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [17]:
# confusion matrix to list 변환
def matrix_to_list(confu_matrix):
    m_list = []
    tn = confu_matrix[0,0]
    fp = confu_matrix[0,1]
    fn = confu_matrix[1,0]
    tp = confu_matrix[1,1]
    m_list.extend([tn, fp, fn, tp])
    return m_list

In [3]:
# 분석용 데이터 입력
stock_name = 'sec'
directory_for_ml = '../../data/data_for_ml/'
fname = f'df_{stock_name}_sel.pkl'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name)

In [4]:
new_columns = ['kospi_cr', 'high', 'financeetc', 'insurance', 'low', 'dji_f_cr', 'weekday', 'vix_cr', 'wti_cr', 'bond_kor_2_cr', 'cr_05']
# new_columns = ['kospi_cr', 'high', 'insurance', 'kosdaq_cr', 'bank', 'bond_usa_10_cr', 'invtrust', 'bond_kor_2_cr', 'corporateetc', 'dji_f_cr', 'cr_05']
new_columns = ['high', 'open', 'kospi_cr', 'low', 'spx_f_cr', 'sox_cr', 'dji_f_cr', 'dji_cr', 'invtrust', 'spx_cr', 'cr_05']

In [5]:
df_new = df[new_columns]

In [12]:
data = df_new.iloc[:240, :-1]
data_test = df_new.iloc[240:, :-1]
target = df_new.iloc[:240, -1]
test_target = df_new.iloc[240:, -1]
# df_sel.tail()
# df_test.head()

In [7]:
train_input, val_input, train_target, val_target = train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

In [10]:
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(data_test)

In [18]:
sc =  SGDClassifier(loss='log_loss', max_iter=2000, random_state=42)
sc.fit(train_scaled, train_target)

train_score_sc = sc.score(train_scaled, train_target)
val_score_sc = sc.score(val_scaled, val_target)
test_score_sc = sc.score(test_scaled, test_target)

print("sgd classifier    train accuracy: {:.4f}, val accuracy: {:.4f}, test accuracy: {:.4f}". 
      format(train_score_sc, val_score_sc, test_score_sc))

cm = matrix_to_list(confusion_matrix(test_target, sc.predict(test_scaled)))
print("cm", cm)

sgd classifier    train accuracy: 0.8594, val accuracy: 0.7708, test accuracy: 0.8621
cm [19, 1, 3, 6]
